In [ ]:
# # ABSTRACTIONS

from abc import ABC, abstractmethod
from dataclasses import dataclass, field
from typing import Any, Optional

import pandas as pd


class State(ABC):
   """
   Abstract base class representing a simulation state in a state machine.
   Each concrete state encapsulates specific logic to update the simulation
   context and decide when to transition to the next state.
   """

   @abstractmethod
   def run_step(self, state_context: dict) -> dict:
       """
       Run a simulation step within this state.

       Args:
           state_context (dict): The current state of the simulation.

       Returns:
           dict: The updated state context after applying this state's logic.
       """
       pass

   @abstractmethod
   def should_transition(self, state_context: dict) -> bool:
       """
       Determine whether the simulation should transition to another state.

       Args:
           state_context (dict): The current state of the simulation.

       Returns:
           bool: True if the simulation should transition to another state.
       """
       pass

   @abstractmethod
   def next_state(self) -> str:
       """
       Define the name of the next state this state transitions to.

       Returns:
           str: The name identifier of the next state.
       """
       pass


class OutputFormatter(ABC):
   """
   Interface for extracting agent-facing output from internal simulation state.
   """

   @abstractmethod
   def extract(self, context: dict) -> dict:
       """
       Extract relevant output fields from internal state.

       Args:
           context (dict): Full simulation state at the current timestep.

       Returns:
           dict: Reduced agent-facing state.
       """
       pass


class AbstractStateMachine(ABC):
   """
   Abstract base class for any finite state machine controlling process simulation.
   """

   @abstractmethod
   def step(self, context: dict) -> dict:
       """
       Advance the simulation one step forward by updating the context.

       Args:
           context (dict): The current simulation context.

       Returns:
           dict: The updated context after processing the current state logic.
       """
       pass


class AbstractProcessSimulator(ABC):
   """
   Abstract base class for running a simulation over time using a state machine.
   """

   @abstractmethod
   def simulate(
       self,
       start_time: pd.Timestamp,
       end_time: pd.Timestamp,
       initial_context: dict,
       modifiers: Optional[list[Any]] = None,
   ) -> Any:
       """
       Execute a full simulation loop over the specified time range.

       Args:
           start_time: Start time of the simulation.
           end_time: End time of the simulation.
           initial_context (dict): Initial simulation state.
           modifiers: Optional list of context-modifying strategies.

       Returns:
           Any: Simulation results.
       """
       pass


class SimulationModifier(ABC):
   """
   Abstract base for all simulation modifiers (noise, agent actions, anomalies).
   """

   @abstractmethod
   def apply(self, context: dict, message: Optional[str]) -> dict:
       """
       Modify the simulation context based on the current timestep.

       Args:
           context (dict): Full simulation state at the current step.

       Returns:
           dict: Modified simulation context.
       """
       pass


In [2]:
# # Concrete Implementations


# +
class ProduceState(State):
   """
   Simulates the water production (filtration) phase.

   In this state:
   - TMP (transmembrane pressure) increases (membranes are fouling).
   - Flow decreases (because resistance is building).
   - Time spent in this mode is tracked with `t_mode`.
   """

   def run_step(self, state_context: dict) -> dict:
       print(f"Running ProduceState at t = {state_context['t_stamp']}")
       state_context["tmp"] += state_context["tmp_growth"]  # Membrane fouling
       state_context["flow"] -= state_context["flow_decay"]  # Flow drops
       state_context["t_mode"] += 1  # Track time in this mode
       state_context["mode"] = "produce"
       return state_context

   def should_transition(self, state_context: dict) -> bool:
       # Transition if TMP too high OR max time in production reached
       if state_context["tmp"] >= state_context["TMP_max"]:
           state_context["transition_reason"] = "TMP limit exceeded"
           return True
       elif state_context["t_mode"] >= state_context["t_produce_max"]:
           state_context["transition_reason"] = "Max production time reached"
           return True
       return False

   def next_state(self) -> str:
       return "flush"


class FlushState(State):
   """
   Simulates a flush/cleaning phase.

   In this state:
   - TMP is reduced (membranes recover performance).
   - Flow is restored.
   - The system resets `t_mode` and prepares to re-enter Produce.
   """

   def run_step(self, state_context: dict) -> dict:
       print(f"Running FlushState at t = {state_context['t_stamp']}")
       state_context["tmp"] = max(state_context["tmp"] - 0.2, 0)  # Clean up TMP
       state_context["flow"] = state_context["initial_flow"]  # Reset flow
       state_context["t_mode"] += 1
       state_context["mode"] = "flush"
       return state_context

   def should_transition(self, state_context: dict) -> bool:
       # Flush for a set time, then go back to production
       if state_context["t_mode"] >= state_context["t_flush_max"]:
           state_context["transition_reason"] = "Flush time completed"
           return True
       return False

   def next_state(self) -> str:
       return "produce"


# -


@dataclass
class AgentActionModifier(SimulationModifier):
   """
   Injects agent-specified parameter updates at specified timestamps.

   Attributes:
       df (pd.DataFrame): A DataFrame where each row contains:
           - a 't_stamp' timestamp column (required)
           - one or more parameter values to inject into the simulation context
   """

   df: pd.DataFrame
   event_map: dict[pd.Timestamp, dict] = field(init=False)

   def __post_init__(self):
       self.event_map = self._build_event_map(self.df)

   def _build_event_map(self, df: pd.DataFrame) -> dict[pd.Timestamp, dict]:
       df = df.copy()
       df["t_stamp"] = pd.to_datetime(df["t_stamp"])
       df = df.set_index("t_stamp")
       return {t: row.dropna().to_dict() for t, row in df.iterrows()}

   def apply(self, context: dict) -> dict:
       t = context["t_stamp"]
       if t in self.event_map:
           print(f"[AGENT ACTION] at {t}: {self.event_map[t]}")
           context.update(self.event_map[t])
       return context


@dataclass
class AnomalyInjector(SimulationModifier):
   """
   Injects anomaly events into the simulation at specific timestamps.

   Attributes:
       df (pd.DataFrame): A DataFrame where each row represents:
           - 't_stamp': timestamp when the anomaly should be injected
           - other fields to override in the simulation context
   """

   df: pd.DataFrame
   event_map: dict[pd.Timestamp, dict] = field(init=False)

   def __post_init__(self):
       self.event_map = self._build_event_map(self.df)

   def _build_event_map(self, df: pd.DataFrame) -> dict[pd.Timestamp, dict]:
       df = df.copy()
       df["t_stamp"] = pd.to_datetime(df["t_stamp"])
       df = df.set_index("t_stamp")
       return {t: row.dropna().to_dict() for t, row in df.iterrows()}

   def apply(self, context: dict) -> dict:
       t = context["t_stamp"]
       if t in self.event_map:
           print(f"[ANOMALY] at {t}: {self.event_map[t]}")
           context.update(self.event_map[t])
       return context


@dataclass
class AgentUFOutputFormatter(OutputFormatter):
   """
   A sample formatter for UF systems. Only outputs the key metrics
   typically needed by an optimizer agent.
   """

   def extract(self, context: dict) -> dict:
       return {
           "t_stamp": context["t_stamp"],
           "mode": context["mode"],
           "tmp": context["tmp"],
           "flow": context["flow"],
       }


@dataclass
class StateMachine(AbstractStateMachine):
   """
   Manages the current state and transitions.

   - Holds a dictionary of all states (keyed by name).
   - Keeps track of the currently active state.
   - Delegates step execution to the current state.
   - Handles state transitions based on each state's logic.

   Attributes:
       states (dict[str, State]): A mapping of state names to State instances.
       current_state (State): The current active state in the state machine.
   """

   states: dict[str, State]
   current_state_name: str

   def __post_init__(self):
       self.current_state = self.states[self.current_state_name]

   def step(self, context: dict) -> dict:
       """
       Executes the current state's logic and handles state transitions.

       Args:
           context (dict): The current simulation context.

       Returns:
           dict: Updated simulation context.
       """
       print(f"\n[STATE MACHINE] Current state: {self.current_state_name}")
       context = self.current_state.run_step(context)

       if self.current_state.should_transition(context):
           reason = context.get("transition_reason", "Unknown reason")
           next_state_name = self.current_state.next_state()
           print(
               f"[TRANSITION] {self.current_state_name} ➡️ {next_state_name}  | Reason: {reason}"
           )
           self.current_state = self.states[next_state_name]
           self.current_state_name = next_state_name
           context["t_mode"] = 0  # Reset time in new mode

       else:
           context["transition_reason"] = None  # No transition this step

       return context


@dataclass
class ProcessSimulator(AbstractProcessSimulator):
   """
   Drives the process simulation loop over time using a state machine and optional modifiers.

   Responsibilities:
   - Advances simulation in fixed time steps (e.g., 1 second per step).
   - Applies modifiers before each step (agent actions, anomalies, noise, etc.).
   - Delegates state logic to the FSM (`state_machine.step()`).
   - Collects both internal state (`df_internal_state`) and agent-facing state (`df_agent`).

   Attributes:
       state_machine (AbstractStateMachine): FSM controlling state transitions and logic.
       output_formatter (OutputFormatter): Strategy to extract agent-facing data from context.
       time_step_s (int): Time increment per step in seconds.
       config (dict): configuration dictionary.
   """

   state_machine: AbstractStateMachine
   output_formatter: AgentUFOutputFormatter
   time_step_s: int = 1
   config: dict = field(init=False)
   modifiers: list[SimulationModifier] = field(init=False, default=None)
   df_internal_state: pd.DataFrame = field(init=False)
   df_agent: pd.DataFrame = field(init=False)

   def simulate(
       self,
       start_time: pd.Timestamp,
       end_time: pd.Timestamp,
       initial_context: dict,
       modifiers: Optional[list[SimulationModifier]] = None,
   ):
       """
       Executes the full simulation from `start_time` to `end_time`.

       Args:
           start_time (pd.Timestamp): The initial timestamp for the simulation.
           end_time (pd.Timestamp): The final timestamp (exclusive).
           initial_context (dict): Starting simulation values.

       Returns:
           pd.DataFrame: Agent-facing simulation output (`df_agent`).
       """
       print("\n=== Starting Simulation ===\n")

       context = initial_context.copy()
       self.initial_context = context
       self.modifiers = modifiers
       context["t_stamp"] = start_time
       internal_state = []
       agent_history = []

       while context["t_stamp"] < end_time:
           print(f"\n[TIMESTEP] t = {context['t_stamp']}")

           # --- Apply Modifiers Before State Logic ---
           if self.modifiers:
               for modifier in self.modifiers:
                   pre_mod = context.copy()
                   context = modifier.apply(context)

                   # Print modifier effect if any keys changed
                   diff = {
                       k: context[k]
                       for k in context
                       if k in pre_mod and context[k] != pre_mod[k]
                   }
                   if diff:
                       print(
                           f"[MODIFIER EFFECT] {modifier.__class__.__name__} applied changes: {diff}"
                       )

           # --- Run State Machine Logic ---
           context = self.state_machine.step(context)

           # --- Record Output ---
           internal_state.append(context.copy())
           agent_history.append(self.output_formatter.extract(context))

           print(f"[UPDATED CONTEXT] → {context}")
           context["t_stamp"] += pd.Timedelta(seconds=self.time_step_s)

       print("\n=== Simulation Complete ===\n")
       self.df_internal_state = pd.DataFrame(internal_state)
       self.df_agent = pd.DataFrame(agent_history)



In [4]:
# # How to Plug It All Together

# === Client-Agnostic Configuration ===
client_config = {
   "TMP_max": 1.5,
   "t_produce_max": 10,
   "t_flush_max": 5,
   "initial_tmp": 0.8,
   "initial_flow": 50.0,
   "flow_decay": 2.5,
   "tmp_growth": 0.1,
}

# +
# === Initialize states and FSM ===
states = {"produce": ProduceState(), "flush": FlushState()}
finite_state_machine = StateMachine(states=states, current_state_name="produce")

# === Initial context based on config ===
initial_context = {
   "tmp": client_config["initial_tmp"],
   "flow": client_config["initial_flow"],
   "initial_flow": client_config["initial_flow"],
   "TMP_max": client_config["TMP_max"],
   "t_produce_max": client_config["t_produce_max"],
   "t_flush_max": client_config["t_flush_max"],
   "tmp_growth": client_config["tmp_growth"],
   "flow_decay": client_config["flow_decay"],
   "t_mode": 0,
}

agent_action_df = pd.DataFrame(
   [
       {"t_stamp": "2025-01-01 00:00:10", "TMP_max": 1.3, "flow_decay": 6.0},
       {"t_stamp": "2025-01-01 00:00:15", "TMP_max": 1.3, "flow_decay": 6.0},
       {"t_stamp": "2025-01-01 00:02:25", "TMP_max": 1.3, "flow_decay": 6.0},
   ]
)

agent_modifier = AgentActionModifier(df=agent_action_df)

anomaly_df = pd.DataFrame(
   [
       {
           "t_stamp": "2025-01-01 00:00:30",
           "tmp": 3.2,
           "transition_reason": "Membrane rupture",
       },
       {
           "t_stamp": "2025-01-01 00:00:45",
           "flow": 0.0,
           "transition_reason": "Valve jammed",
       },
   ]
)

anomaly_modifier = AnomalyInjector(df=anomaly_df)




In [ ]:
# === Run the simulation ===
# Create formatter
formatter = AgentUFOutputFormatter()


sim = ProcessSimulator(
   state_machine=finite_state_machine,
   output_formatter=formatter,
   time_step_s=5,
)
sim.simulate(
   start_time=pd.Timestamp("2025-01-01 00:00:00"),
   end_time=pd.Timestamp("2025-01-01 00:10:00"),
   initial_context=initial_context,
   modifiers=[agent_modifier, anomaly_modifier],
)

df = sim.df_internal_state
df_agent = sim.df_agent
# -

df.head(20)

df.tail(20)

df_agent.head(10)

df_agent.tail(10)


@dataclass
class AgentActionModifier(SimulationModifier):
   """
   Injects agent-specified parameter updates at specified timestamps.

   Attributes:
       df (pd.DataFrame): A DataFrame where each row contains:
           - a 't_stamp' timestamp column (required)
           - one or more parameter values to inject into the simulation context
   """

   df: pd.DataFrame
   time_stamp_column: str
   event_map: dict[pd.Timestamp, dict] = field(init=False)

   def __post_init__(self):
       self.event_map = self._build_event_map(self.df)

   def _build_event_map(self, df: pd.DataFrame) -> dict[pd.Timestamp, dict]:
       df = df.copy()
       df[self.time_stamp_column] = pd.to_datetime(df[self.time_stamp_column])
       df = df.set_index(self.time_stamp_column)
       return {time_stamp: row.dropna().to_dict() for time_stamp, row in df.iterrows()}

   def apply(self, context: dict, message: Optional[str]) -> dict:
       time_stamp_to_check = context[self.time_stamp_column]
       if time_stamp_to_check in self.event_map:
           print(
               f"[AGENT ACTION] at {time_stamp_to_check}: {self.event_map[time_stamp_to_check]}"
           )
           context.update(self.event_map[time_stamp_to_check])
           context["message"] = message
           print(context)
       return context


# +
agent_action_df = pd.DataFrame(
   [
       {"t_stamp": "2025-01-01 00:00:10", "TMP_max": 1.3, "flow_decay": 6.0},
       {"t_stamp": "2025-01-01 00:00:15", "TMP_max": 1.3, "flow_decay": 6.0},
       {"t_stamp": "2025-01-01 00:02:25", "TMP_max": 1.3, "flow_decay": 6.0},
   ]
)

agent_modifier = AgentActionModifier(df=agent_action_df, time_stamp_column="t_stamp")
# -

initial_context_test = {
   "tmp": client_config["initial_tmp"],
   "flow": client_config["initial_flow"],
   "initial_flow": client_config["initial_flow"],
   "TMP_max": client_config["TMP_max"],
   "t_produce_max": client_config["t_produce_max"],
   "t_flush_max": client_config["t_flush_max"],
   "tmp_growth": client_config["tmp_growth"],
   "flow_decay": client_config["flow_decay"],
   "t_mode": 0,
   "t_stamp": pd.Timestamp("2025-01-01 00:00:15"),
}

agent_modifier.event_map

agent_modifier.apply(
   context=initial_context_test,
   message="adding agent set point",
)


=== Starting Simulation ===


[TIMESTEP] t = 2025-01-01 00:00:00

[STATE MACHINE] Current state: produce
Running ProduceState at t = 2025-01-01 00:00:00
[UPDATED CONTEXT] → {'tmp': 0.9, 'flow': 47.5, 'initial_flow': 50.0, 'TMP_max': 1.5, 't_produce_max': 10, 't_flush_max': 5, 'tmp_growth': 0.1, 'flow_decay': 2.5, 't_mode': 1, 't_stamp': Timestamp('2025-01-01 00:00:00'), 'mode': 'produce', 'transition_reason': None}

[TIMESTEP] t = 2025-01-01 00:00:05

[STATE MACHINE] Current state: produce
Running ProduceState at t = 2025-01-01 00:00:05
[UPDATED CONTEXT] → {'tmp': 1.0, 'flow': 45.0, 'initial_flow': 50.0, 'TMP_max': 1.5, 't_produce_max': 10, 't_flush_max': 5, 'tmp_growth': 0.1, 'flow_decay': 2.5, 't_mode': 2, 't_stamp': Timestamp('2025-01-01 00:00:05'), 'mode': 'produce', 'transition_reason': None}

[TIMESTEP] t = 2025-01-01 00:00:10
[AGENT ACTION] at 2025-01-01 00:00:10: {'TMP_max': 1.3, 'flow_decay': 6.0}
[MODIFIER EFFECT] AgentActionModifier applied changes: {'TMP_max': 1.3, 'flow_d

{'tmp': 0.8,
 'flow': 50.0,
 'initial_flow': 50.0,
 'TMP_max': 1.3,
 't_produce_max': 10,
 't_flush_max': 5,
 'tmp_growth': 0.1,
 'flow_decay': 6.0,
 't_mode': 0,
 't_stamp': Timestamp('2025-01-01 00:00:15'),
 'message': 'adding agent set point'}